In [1]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from test_db_schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [4]:
from typing import Any


async def dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    match_by=User.name,
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [7]:
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(schema=TestSchema)
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

In [8]:
db[User].values()


[
    User({'name': 'John', 'age': 31, '_id': 'ed5a3438-1249-461e-a381-53790e04a962'}),
    User({'name': 'Jane', 'age': 56, '_id': 'a9243cc6-1a7c-479b-a857-2eb6cc7d1a5a'}),
    User({'name': 'Jack', 'age': 66, '_id': '29df440f-eec0-4e42-84f5-542ea48f51c1'})
]

In [9]:
db[Search.term].values()

['test']

In [10]:
list(db[Search.result_count].items())

[('test', 3)]

In [11]:
list(db[Search.results].items())


[
    (
        ('test', 1),
        Project({'number': 1, 'name': 'cleaning shoes', 'start': datetime.date(2020, 1, 2), 'end': datetime.date(2020, 1, 5), 'status': 'done'})
    ),
    (
        ('test', 2),
        Project({'number': 2, 'name': 'fixing cars', 'start': datetime.date(2020, 1, 3), 'end': datetime.date(2020, 1, 6), 'status': 'started'})
    ),
    (
        ('test', 0),
        Project({'number': 0, 'name': 'baking cake', 'start': datetime.date(2020, 1, 1), 'end': datetime.date(2020, 1, 4), 'status': 'done'})
    )
]

In [12]:
list(db[Search.results].items())[0][1].start.year

2020

In [13]:
db[Search.results].df()

term,test_db_schema_Project_number,number,test_db_schema_Organization__id,name,status,start,end
str,i64,i64,str,str,str,date,date
"""test""",1,1,"""003c5f1f-52ba-494b-b892-dc9d9a…","""cleaning shoes""","""done""",2020-01-02,2020-01-05
"""test""",2,2,"""da577882-0f69-403b-b9fd-b4b24e…","""fixing cars""","""started""",2020-01-03,2020-01-06
"""test""",0,0,"""7fe5f55f-6708-4e01-af9a-52b9fe…","""baking cake""","""done""",2020-01-01,2020-01-04


In [14]:
db[Organization].df()

_id,address,city,name
str,str,str,str
"""003c5f1f-52ba-494b-b892-dc9d9a…","""Main Street 1""","""Bakerville""","""Bakery"""
"""7fe5f55f-6708-4e01-af9a-52b9fe…","""Main Street 2""","""Shoetown""","""Shoe Shop"""
"""da577882-0f69-403b-b9fd-b4b24e…","""Main Street 3""","""Cartown""","""Car Shop"""


In [15]:
db.describe()


{
    'schema': {
        'package': 'py-research',
        'module': 'test_db_schema',
        'object': 'TestSchema',
        'package_version': '^5.1.1.dev133+gf6dad64.d20250116',
        'repo': 'https://github.com/cloudlane-one/py-research.git',
        'repo_revision': 'fd6e200126d1d7784e30e4bf905db2fb1df3ef20',
        'module_path': WindowsPath('exp/test_db_schema.py'),
        'module_dirty': False,
        'docs_url': 'https://cloudlane-one.github.io/py-research/'
    },
    'contents': {
        'records': {
            'test_db_schema.Search': 1,
            'test_db_schema.Organization': 3,
            'test_db_schema.Project': 3,
            'test_db_schema.User': 3,
            'test_db_schema.Task': 6
        },
        'arrays': {'test_db_schema_Organization_countries': 0},
        'relations': {'test_db_schema.SearchResult': 3, 'test_db_schema.Membership': 7}
    },
    'backend': None
}

In [21]:
db[SearchResult._table[SearchResult.score.isin(slice(0.81, 0.9))]].df()

_id,test_db_schema_Search_term,test_db_schema_Project_number,score
str,str,i64,f64
"""6880419913""","""test""",1,0.9


In [17]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

 542317 function calls (508259 primitive calls) in 0.768 seconds

   Ordered by: cumulative time
   List reduced from 2272 to 145 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       52    0.000    0.000    0.275    0.005 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:688(_load_rec_from_item)
       52    0.003    0.000    0.273    0.005 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:567(_load_record)
       23    0.000    0.000    0.262    0.011 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1449(__ior__)
       23    0.002    0.000    0.262    0.011 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2895(_mutate)
       56    0.000    0.000    0.258    0.005 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:829(load)
   168/57    0.001    0.000    0.245    0.004 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:656(_lo